In [2]:
os.environ["OPENAI_API_KEY"] = ""
# OPENAI_API_KEY = ""

In [21]:
! pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 28.8 MB/s eta 0:00:00


In [33]:
from abc import ABC, abstractmethod
from typing import List
from langchain.schema import Document
import openai
import os
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator

# used to create the retriever
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# used to create the retriever
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# used to create the retrieval tool
from langchain.agents import tool

# used to create the memory
from langchain.memory import ConversationBufferMemory

# used to create the agent executor
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor

# used to create the prompt template
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema import SystemMessage
from langchain.prompts import MessagesPlaceholder

In [30]:
os.environ["OPENAI_API_KEY"] = ""
# This is needed for both the memory and the prompt
memory_key = "history"

In [12]:
loader = TextLoader('Data.txt', encoding='utf8')
data = loader.load()

In [15]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(data)

In [17]:
embeddings = OpenAIEmbeddings()

In [22]:
db = FAISS.from_documents(texts, embeddings)

In [23]:
retriever = db.as_retriever()

In [26]:
@tool
def tool(query):
    "Searches and returns documents regarding the llm powered autonomous agents blog"
    docs = retriever.get_relevant_documents(query)
    return docs

tools = [tool]

In [31]:
memory = ConversationBufferMemory(memory_key=memory_key, return_messages=True)

In [49]:
system_message = SystemMessage(
        content=(
            "Ты помощник Ашана. Старайся отвечать на все поставленные вопросы. "
            "Для этого в первую очередь используй предоставленные данные Ашана."
        )
)

In [50]:
prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)]
    )

In [51]:
llm = ChatOpenAI(temperature = 0)

agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)

In [52]:
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

# Test

In [59]:
result = agent_executor({"input": "I'm 14"})

result["output"]



> Entering new AgentExecutor chain...
Hello, Michael! How can I assist you today? Is there something specific you would like to know or discuss?

> Finished chain.


'Hello, Michael! How can I assist you today? Is there something specific you would like to know or discuss?'

In [60]:
result = agent_executor({"input": "Сколько мне лет?"})

result["output"]



> Entering new AgentExecutor chain...
Вы сказали, что вам 14 лет.

> Finished chain.


'Вы сказали, что вам 14 лет.'

In [55]:
result = agent_executor({"input": "В какое время работают магазины Ашана? Мне нужно точное время"})

result["output"]



> Entering new AgentExecutor chain...

Invoking: `tool` with `{'query': 'working hours of Ashan stores'}`


[Document(page_content='Могу ли я сделать заказ, если у меня нет карты лояльности?\nЗаказ может оформить только владелец карты АШАН, зарегистрированный в личном кабинете на сайте или в мобильном приложении Мой АШАН. Как получить электронную карту и зарегистрировать личный кабинет можно посмотреть здесь.\n\nКак войти в свою учетную запись?\nДля защиты ваших персональных данных и бонусного баланса авторизация теперь доступна только по номеру телефона.\n\nВведите номер телефона, который вы привязали к учетной записи и нажмите на экране «Получить код». Дождитесь смс с кодом и введите его в соответствующее поле.\n\nКак мне воспользоваться социальными привилегиями при заказе в интернет-магазине?\nВам необходимо оформить социальный статус вашей карты лояльности. После того как подтвердится ее статус, укажите номер вашей карты при оформлении заказа в нашем интернет-магазине и получите 

'Магазины Ашана имеют различное время работы в зависимости от конкретного магазина и его расположения. Обычно магазины Ашана открыты с утра до вечера, с понедельника по воскресенье. Однако, рекомендуется проверить рабочие часы конкретного магазина на официальном сайте Ашана или связаться с их службой поддержки для получения точной информации о времени работы выбранного вами магазина.'

In [57]:
result = agent_executor({"input": "Как мне воспользоваться социальными привилегиями при заказе в интернет-магазине?"})

result["output"]



> Entering new AgentExecutor chain...
Для использования социальных привилегий при заказе в интернет-магазине Ашана, вам может потребоваться выполнить следующие шаги:

1. Зарегистрируйтесь на сайте Ашана или в их приложении, если у вас еще нет аккаунта.
2. Проверьте, какие социальные привилегии доступны в вашем регионе и для вашей группы лиц (например, пенсионеры, инвалиды, многодетные семьи и т. д.).
3. Убедитесь, что у вас есть необходимые документы или идентификационные данные, которые подтверждают вашу принадлежность к соответствующей группе лиц.
4. При оформлении заказа в интернет-магазине Ашана, укажите свои социальные привилегии или выберите соответствующую опцию, если она доступна.
5. При необходимости, предоставьте дополнительные документы или информацию, чтобы подтвердить вашу принадлежность к социальной группе.
6. Проверьте, что социальные привилегии были применены к вашему заказу и отражены в итоговой стоимости или условиях доставки.
7. Продолжайте оформление заказа и выбе

'Для использования социальных привилегий при заказе в интернет-магазине Ашана, вам может потребоваться выполнить следующие шаги:\n\n1. Зарегистрируйтесь на сайте Ашана или в их приложении, если у вас еще нет аккаунта.\n2. Проверьте, какие социальные привилегии доступны в вашем регионе и для вашей группы лиц (например, пенсионеры, инвалиды, многодетные семьи и т. д.).\n3. Убедитесь, что у вас есть необходимые документы или идентификационные данные, которые подтверждают вашу принадлежность к соответствующей группе лиц.\n4. При оформлении заказа в интернет-магазине Ашана, укажите свои социальные привилегии или выберите соответствующую опцию, если она доступна.\n5. При необходимости, предоставьте дополнительные документы или информацию, чтобы подтвердить вашу принадлежность к социальной группе.\n6. Проверьте, что социальные привилегии были применены к вашему заказу и отражены в итоговой стоимости или условиях доставки.\n7. Продолжайте оформление заказа и выберите удобный способ оплаты и до